# Predicting bike sharing data

In this project, we'll build a neural network and use it to predict daily bike rental ridership.

## Index
- [Load and prepare the data](#load-data)
- [Checking out the data](#check-data)
- [Preparing the data](#prepare-data)
- [Building the network](#build-network)
- [Training the network](#train-network)
- [Checking out predictions](#predictions)